# VacationPy
----

#### Objectives:
1. Create a **Humidity Heatmap** using data extracted from *OpenWeather API and Google Maps JS API.*
2. Using the weather details, find the **best weather conditions and locate the closest hotels** to them using *Google Places API.*


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_w_df = pd.read_csv('output_data/cities.csv')

In [3]:
cities_w_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,charters towers,15,AU,1603323403,45,-20.10,146.27,84.22,11.92
1,1,yola,30,NG,1603323472,64,9.20,12.48,77.59,3.91
2,2,jacmel,75,HT,1603323211,83,18.23,-72.53,78.80,5.82
3,3,pec,36,XK,1603323473,46,42.66,20.29,37.40,1.90
4,4,vakhrushevo,40,RU,1603323473,80,55.23,61.72,30.20,8.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)
#Create a list with locations
locations = cities_w_df[["Lat", "Lng"]]
#Get humidity series
humidity =cities_w_df['Humidity'].astype(float)
max_humidity = cities_w_df['Humidity'].max().astype(float)

In [5]:
#Build map
fig = gmaps.figure()
#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=2)
#Add locations and humidity layer
fig.add_layer(heat_layer)
#Display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Set best weather conditions
low_t=70
hihg_t=80
wind_s=10
#Create dataframe with best weather cities
best_w_df=cities_w_df.loc[(cities_w_df['Max Temp']>low_t) & (cities_w_df['Max Temp']<hihg_t) & (cities_w_df['Wind Speed']<wind_s) & (cities_w_df['Cloudiness']==0)]
best_w_df=best_w_df.reset_index()
best_w_df

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,54,57,aswan,0,EG,1603323493,28,24.09,32.91,76.14,7.70
1,57,60,namwala,0,ZM,1603323494,31,-15.75,26.44,73.38,6.33
2,105,112,napasar,0,IN,1603323513,32,27.97,73.55,71.42,2.42
3,110,117,gat,0,IL,1603323245,73,31.61,34.76,73.00,5.79
4,115,122,safaga,0,EG,1603323516,57,26.73,33.94,77.00,4.70
5,208,224,buin,0,CL,1603323553,52,-33.73,-70.75,71.60,9.17
6,235,257,esna,0,EG,1603323565,29,25.29,32.55,77.00,3.36
7,416,452,harpalpur,0,IN,1603323636,28,25.29,79.33,72.43,1.88
8,420,456,maragogi,0,BR,1603323638,86,-9.01,-35.22,76.82,9.06
9,422,458,palestine,0,PS,1603323639,77,31.92,35.20,73.00,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Function to get data from Google Nearby Serach API
def getHotel(lat,lng):
    params = {
        "keyword":'hotel',
        'types':'hotel',
        'radius':5000,
        'key':g_key,
        'location':f'{lat}, {lng}'
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    return response

In [8]:
#Create hotel_df to capture results from API
hotel_df=[]

#Iterate in df and call getHotel function
for i, row in best_w_df.iterrows():
    function_response = getHotel(row['Lat'],row['Lng'])
    hotel_df.append(function_response)


In [9]:
#Insert into best wheather data frame results from API
best_w_df['Hotel Name'] = ""

#Iterate into restults from API and fill out Hotel Name column
for i, row in best_w_df.iterrows():
    try:
        best_w_df.loc[i,'Hotel Name']=hotel_df[i]['results'][0]['name']
    except:
        print(f"Missing results for city: {row['City']}")

#Remove Cities with no nearby hotels
best_w_df=best_w_df.loc[best_w_df['Hotel Name']!='']
best_w_df.head()


Missing results for city: namwala
Missing results for city: napasar


,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,54,57,aswan,0,EG,1603323493,28,24.09,32.91,76.14,7.70,Sofitel Legend Old Cataract Aswan
3,110,117,gat,0,IL,1603323245,73,31.61,34.76,73.00,5.79,Desert Gat
4,115,122,safaga,0,EG,1603323516,57,26.73,33.94,77.00,4.70,Dolphin Hotel
5,208,224,buin,0,CL,1603323553,52,-33.73,-70.75,71.60,9.17,Motel INSIDE
6,235,257,esna,0,EG,1603323565,29,25.29,32.55,77.00,3.36,Orient Star Dahabyia Isna 05 Nights Each Sunday


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in best_w_df.iterrows()]
hotel_locations = best_w_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)

# Display Map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))